In [16]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [17]:
THRESH_SPO2_LOW = 92
THRESH_SPO2_CRITICAL = 88
THRESH_SHOCK_INDEX_WARNING = 0.9
THRESH_SHOCK_INDEX_CRITICAL = 1.0
THRESH_MAP_LOW = 65  # Mean Arterial Pressure threshold for shock
THRESH_LACTATE_HIGH = 2.0  # mmol/L for tissue hypoperfusion
THRESH_LACTATE_CRITICAL = 4.0
THRESH_UO_LOW = 0.5  # mL/kg/hr (oliguria)

# Trend Analysis
TREND_WINDOW = 6  # Number of readings for short-term trend analysis


# AGE-SPECIFIC VITAL SIGN THRESHOLDS (Low, Normal, High)

AGE_THRESHOLDS = {
    'neonate': {
        'rr_low': 30, 'rr_normal': 40, 'rr_high': 60,
        'hr_low': 100, 'hr_normal': 140, 'hr_high': 160,
        'sbp_low': 60, 'sbp_normal': 70, 'sbp_high': 90,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'infant': {
        'rr_low': 24, 'rr_normal': 30, 'rr_high': 40,
        'hr_low': 80, 'hr_normal': 120, 'hr_high': 140,
        'sbp_low': 70, 'sbp_normal': 85, 'sbp_high': 100,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'child': {
        'rr_low': 16, 'rr_normal': 20, 'rr_high': 30,
        'hr_low': 70, 'hr_normal': 90, 'hr_high': 110,
        'sbp_low': 80, 'sbp_normal': 95, 'sbp_high': 110,
        'temp_low': 36.0, 'temp_normal': 37.0, 'temp_high': 38.0
    },
    'adolescent': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 75, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 105, 'sbp_high': 120,
        'temp_low': 35.8, 'temp_normal': 36.8, 'temp_high': 37.8
    },
    'adult': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 80, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 115, 'sbp_high': 130,
        'temp_low': 35.5, 'temp_normal': 36.8, 'temp_high': 38.0
    },
    'geriatric': {  # Added for elderly patients who may have different baselines
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 24, # Often higher RR baseline
        'hr_low': 55, 'hr_normal': 70, 'hr_high': 90, # Often lower HR
        'sbp_low': 90, 'sbp_normal': 125, 'sbp_high': 140, # Often higher SBP
        'temp_low': 35.5, 'temp_normal': 36.5, 'temp_high': 37.5 # Often lower temp
    }
}

#--- HF-specific thresholds 
HF_RR_SLOPE_BPM_PER_HR = 1.0           # slope threshold (bpm / hour)
HF_SPO2_BURDEN_PCT_24H = 0.30          # fraction of readings <92 in 24h to flag burden
HF_SPO2_LOW = 92
HF_SPO2_CRITICAL = 88
HF_RR_TACHY = 20                       # absolute tachypnea threshold (bpm)
HF_RR_EMERGENT = 30
HF_SBP_HYPOTENSION = 90                # SBP < 90 urgent
HF_SBP_RISE_SLOPE_24H = 5.0            # mmHg per 24h -> small positive weight
HF_SLOPE_MIN_READINGS = 3              # min points to compute slope
HF_SPO2_CRITICAL_SUSTAINED_MINUTES = 5 # sustained low SpO2 window for critical override

# safety for shock index naming compatibility
THRESH_SHOCK_INDEX_HIGH = THRESH_SHOCK_INDEX_CRITICAL if 'THRESH_SHOCK_INDEX_CRITICAL' in globals() else 1.0

# Score weights (tunable)
HF_SCORE_WEIGHTS = {
    "rr_slope": 3,
    "rr_tachy": 2,
    "spo2_burden": 3,
    "spo2_critical_sustained": 4,
    "sbp_hypotension": 4,
    "sbp_rising_trend": 2,
    "shock_index_critical": 3,
    "rr_emergent_with_low_spo2": 4
}

# Score bins -> action
HF_SCORE_ACTIONS = [
    (8, "URGENT: high risk — immediate escalation / ED contact"),
    (5, "High risk: clinician review same day"),
    (3, "Early warning: contact HF clinic / consider diuretic review"),
    (0, "Routine monitoring")
]

In [18]:
def assign_age_category(df):
    """
    Assigns an age category based on the 'age' column in the DataFrame.
    Now includes more granular pediatric categories and a geriatric category.
    """
    df = df.copy()
    
    def _categorize(age):
        if age <= 0.083: return 'neonate'     # < 1 month
        elif age <= 1:   return 'infant'      # 1 month - 1 year
        elif age < 5:    return 'child'       # 1 - 5 years
        elif age < 13:   return 'adolescent'  # 5 - 12 years
        elif age < 65:   return 'adult'       # 13 - 64 years
        else:            return 'geriatric'   # 65+ years
    
    if 'age' in df.columns:
        df['age_category'] = df['age'].apply(_categorize)
    else:
        # Default to adult if age is not provided
        df['age_category'] = 'adult'
    
    return df

In [19]:
def apply_vital_range_flags(df):
    """
    Applies age-specific thresholds to flag abnormal vital signs.
    Now includes flags for all parameters needed across pipelines.
    """
    df = df.copy()
    df = assign_age_category(df)  # Inject age group

    # SpO₂ flags (Absolute threshold)
    df['flag_spo2_low'] = df['spo2'] < THRESH_SPO2_LOW
    df['flag_spo2_critical'] = df['spo2'] < THRESH_SPO2_CRITICAL

    # Temperature flags
    df['flag_temp_high'] = df.apply(lambda row: row['temperature'] >= AGE_THRESHOLDS[row['age_category']]['temp_high'], axis=1)
    df['flag_temp_low'] = df.apply(lambda row: row['temperature'] < AGE_THRESHOLDS[row['age_category']]['temp_low'], axis=1)

    # Respiratory Rate flags
    df['flag_rr_low'] = df.apply(lambda row: row['resp_rate'] < AGE_THRESHOLDS[row['age_category']]['rr_low'], axis=1)
    df['flag_rr_high'] = df.apply(lambda row: row['resp_rate'] >= AGE_THRESHOLDS[row['age_category']]['rr_high'], axis=1)

    # Heart Rate flags
    df['flag_hr_low'] = df.apply(lambda row: row['heart_rate'] < AGE_THRESHOLDS[row['age_category']]['hr_low'], axis=1)
    df['flag_hr_high'] = df.apply(lambda row: row['heart_rate'] >= AGE_THRESHOLDS[row['age_category']]['hr_high'], axis=1)

    # Calculate Shock Index (handle division by zero)
    df['shock_index'] = df['heart_rate'] / np.clip(df['sbp'], a_min=1, a_max=None)    
    # Flag based on Shock Index
    df['flag_si_warning'] = df['shock_index'] >= THRESH_SHOCK_INDEX_WARNING
    df['flag_si_critical'] = df['shock_index'] >= THRESH_SHOCK_INDEX_CRITICAL

    # Blood Pressure flags
    df['flag_sbp_low'] = df.apply(lambda row: row['sbp'] < AGE_THRESHOLDS[row['age_category']]['sbp_low'], axis=1)
    df['flag_sbp_high'] = df.apply(lambda row: row['sbp'] >= AGE_THRESHOLDS[row['age_category']]['sbp_high'], axis=1)
    df['flag_dbp_low'] = df.apply(lambda row: row['dbp'] < (AGE_THRESHOLDS[row['age_category']]['sbp_low'] * 0.6), axis=1) # Estimate DBP low
    df['flag_dbp_high'] = df.apply(lambda row: row['dbp'] >= (AGE_THRESHOLDS[row['age_category']]['sbp_high'] * 0.6), axis=1) # Estimate DBP high

    return df

In [20]:
def compute_recent_trends_delta(df):
    """
    Computes recent trends for each vital by differencing consecutive readings.
    Uses age-specific thresholds + detects likely false-positive variations
    (e.g., transient spikes/drops or conflicting multi-vital patterns).
    """
    df = df.copy().sort_values("timestamp").reset_index(drop=True)

    if 'age_category' not in df.columns:
        df = assign_age_category(df)

    trends = {}
    recent = df.tail(TREND_WINDOW)
    if recent.empty:
        return trends

    age_group = recent['age_category'].iloc[-1]
    thresholds = AGE_THRESHOLDS[age_group]

    vital_map = {
        'rr': ('rr_low', 'rr_normal', 'rr_high'),
        'hr': ('hr_low', 'hr_normal', 'hr_high'), 
        'sbp': ('sbp_low', 'sbp_normal', 'sbp_high'),
        'temperature': ('temp_low', 'temp_normal', 'temp_high'),
        'spo2': (None, None, None)
    }

    # --- Enhanced false positive detection ---
    def is_transient_spike(values, delta, threshold_ratio=0.15):
        """Detect short-lived sharp deviations likely due to artifacts."""
        if len(values) < 3 or np.isnan(values).any():
            return False
        median_val = np.median(values)
        if median_val == 0:
            return False
        deviation = abs(values[-1] - median_val)
        return deviation / median_val > threshold_ratio and abs(delta) < 0.2 * deviation

    def is_unstable_signal(values, threshold_ratio=0.5):
        """Detect excessive variability suggesting measurement noise."""
        if len(values) < 2 or np.isnan(values).any():
            return False
        value_std = np.std(values)
        if value_std == 0:
            return False
        diff_std = np.std(np.diff(values))
        return diff_std > threshold_ratio * value_std

    for vital in ['rr', 'hr', 'sbp', 'temperature', 'spo2']:
        if vital not in recent.columns or recent[vital].isnull().all():
            continue

        y = recent[vital].dropna().values
        if len(y) < 2:
            continue

        avg_delta = np.mean(np.diff(y))
        latest = y[-1]
        trends[f"{vital}_trend"] = round(avg_delta, 3)

        # --- Enhanced false positive detection ---
        transient_spike = is_transient_spike(y, avg_delta)
        unstable_signal = is_unstable_signal(y)
        
        fp_evidence = []
        if transient_spike:
            fp_evidence.append("transient_spike")
        if unstable_signal:
            fp_evidence.append("unstable_signal")

        # SPO₂ special handling
        if vital == 'spo2':
            if latest < THRESH_SPO2_LOW:
                if avg_delta > 0:
                    flag = "Still abnormal — but improving"
                elif avg_delta < 0:
                    flag = "Abnormal and worsening"
                else:
                    flag = "Abnormal and flat"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        else:
            low_key, norm_key, high_key = vital_map[vital]
            low = thresholds[low_key]
            normal = thresholds[norm_key]  
            high = thresholds[high_key]

            if latest < low or latest > high:
                if (latest > high and avg_delta < 0) or (latest < low and avg_delta > 0):
                    flag = "Still abnormal — but improving"
                else:
                    flag = "Abnormal and worsening"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        if fp_evidence:
            flag += f" (possible false-positive: {', '.join(fp_evidence)})"
            trends[f"{vital}_false_positive"] = True
            trends[f"{vital}_fp_evidence"] = fp_evidence
            trends[f"{vital}_confidence"] = "LOW"
        else:
            trends[f"{vital}_false_positive"] = False
            trends[f"{vital}_confidence"] = "HIGH"

        trends[f"{vital}_trend_flag"] = flag

    # --- Shock Index trend ---
    if all(col in recent.columns for col in ['hr', 'sbp']):
        hr = recent['hr'].values
        sbp = np.clip(recent['sbp'].values, a_min=1, a_max=None)
        si = hr / sbp

        if len(si) >= 2:
            avg_si_delta = np.mean(np.diff(si))
            trends['shock_index_trend'] = round(avg_si_delta, 3)

            latest_si = si[-1]
            if latest_si >= THRESH_SHOCK_INDEX_CRITICAL:
                flag = "Shock Index high — improving" if avg_si_delta < 0 else "Shock Index high — worsening"
            else:
                flag = "Normal but improving" if avg_si_delta < 0 else "Normal but rising"

            si_fp_evidence = []
            if is_unstable_signal(si, threshold_ratio=0.3):
                si_fp_evidence.append("unstable_si")

            if latest_si >= THRESH_SHOCK_INDEX_CRITICAL:
                latest_hr = recent['hr'].iloc[-1]
                latest_sbp = recent['sbp'].iloc[-1]
                hr_normal = latest_hr < thresholds['hr_high']
                sbp_normal = latest_sbp >= thresholds['sbp_low']
                
                if (hr_normal and not sbp_normal) or (sbp_normal and not hr_normal):
                    si_fp_evidence.append("single_component_abnormality")

            if si_fp_evidence:
                flag += f" (possible false-positive: {', '.join(si_fp_evidence)})"
                trends['shock_index_false_positive'] = True
                trends['shock_index_fp_evidence'] = si_fp_evidence
                trends['shock_index_confidence'] = "LOW"
            else:
                trends['shock_index_false_positive'] = False  
                trends['shock_index_confidence'] = "HIGH"

            trends['shock_index_trend_flag'] = flag

    # --- Overall confidence summary ---
    false_positive_count = sum(1 for key in trends if key.endswith('_false_positive') and trends[key])
    total_metrics = sum(1 for key in trends if key.endswith('_false_positive'))
    
    if total_metrics > 0:
        fp_ratio = false_positive_count / total_metrics
        if fp_ratio >= 0.5:
            trends['overall_confidence'] = "LOW"
        elif fp_ratio >= 0.25:
            trends['overall_confidence'] = "MEDIUM" 
        else:
            trends['overall_confidence'] = "HIGH"
    else:
        trends['overall_confidence'] = "HIGH"

    return trends


In [21]:
def _ensure_sorted_index(df):
    df = df.copy()
    if 'timestamp' not in df.columns:
        raise ValueError("DataFrame must contain 'timestamp' column.")
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df.sort_values('timestamp').reset_index(drop=True)


In [22]:
def _compute_time_slope(series_vals, series_times):
    """
    Compute slope per hour using linear fit (units = value per hour).
    series_times: pandas Series of timestamps
    series_vals: numpy array or Series of numeric values
    Returns slope (float) in units per hour. If not computable, returns np.nan.
    """
    if len(series_vals) < HF_SLOPE_MIN_READINGS:
        return np.nan
    
    # Convert to numpy arrays to avoid pandas index issues
    series_vals_np = np.array(series_vals, dtype=float)
    
    # Convert timestamps to hours relative to first time
    # FIX: Use .values to get numpy array and avoid index issues
    series_times_np = pd.to_datetime(series_times).values
    t_seconds = series_times_np.astype('int64') // 1_000_000_000  # Convert to seconds since epoch
    
    # If all times identical, avoid polyfit error
    if np.all(t_seconds == t_seconds[0]):
        return np.nan
    
    try:
        t_hours = (t_seconds - t_seconds[0]) / 3600.0
        slope, intercept = np.polyfit(t_hours, series_vals_np, 1)
        return float(slope)  # units per hour
    except Exception:
        return np.nan


def _percent_below(series, threshold):
    arr = series.dropna().values
    if len(arr) == 0:
        return 0.0
    return float((arr < threshold).sum()) / float(len(arr))

In [23]:
def _sustained_low_event(series_spo2, series_times, thresh, min_minutes):
    """
    Check if there exists any sustained interval where SpO2 <= thresh
    for at least min_minutes. series_times should be pandas timestamps aligned with series_spo2.
    """
    if len(series_spo2) == 0:
        return False
    df = pd.DataFrame({"spo2": series_spo2.values, "timestamp": pd.to_datetime(series_times.values)})
    df = df.reset_index(drop=True)
    df['is_low'] = df['spo2'] <= thresh
    if not df['is_low'].any():
        return False

    # group consecutive low readings
    groups = (df['is_low'] != df['is_low'].shift()).cumsum()
    grouped = df.groupby(groups)
    for _, g in grouped:
        if not g['is_low'].iloc[0]:
            continue
        duration = g['timestamp'].iloc[-1] - g['timestamp'].iloc[0]
        if duration >= timedelta(minutes=min_minutes):
            return True
    return False

In [24]:
def hf_decompensation_assessment(df_patient, window_hours=24):
    """
    Assess HF decompensation using only continuous vital signs.
    No weight data required.
    """
    df = _ensure_sorted_index(df_patient)
    now = df['timestamp'].iloc[-1]

    # subset window
    window_start = now - pd.Timedelta(hours=window_hours)
    window_df = df[df['timestamp'] >= window_start].copy()
    if window_df.empty:
        raise ValueError("No data in the lookback window to compute HF assessment.")

    # ensure age_category exists
    if 'age_category' not in df.columns:
        try:
            df = assign_age_category(df)
            window_df = df[df['timestamp'] >= window_start]
        except Exception:
            df['age_category'] = 'adult'
            window_df['age_category'] = 'adult'

    flags = {}
    trends = {}

    # ---- SpO2 burden & critical sustained ----
    spo2_series = window_df['spo2'].dropna()
    trends['spo2_burden92_frac_24h'] = _percent_below(spo2_series, HF_SPO2_LOW)
    flags['spo2_burden'] = trends['spo2_burden92_frac_24h'] >= HF_SPO2_BURDEN_PCT_24H
    flags['spo2_critical_sustained'] = _sustained_low_event(window_df['spo2'], window_df['timestamp'],
                                                             HF_SPO2_CRITICAL, HF_SPO2_CRITICAL_SUSTAINED_MINUTES)

    # ---- RR mean and slope ----
    rr_series = window_df['resp_rate'].dropna()
    trends['rr_mean_24h'] = float(rr_series.mean()) if len(rr_series) > 0 else np.nan
    trends['rr_slope_bph'] = _compute_time_slope(rr_series.values, window_df.loc[rr_series.index, 'timestamp'])
    flags['rr_slope'] = not np.isnan(trends['rr_slope_bph']) and trends['rr_slope_bph'] >= HF_RR_SLOPE_BPM_PER_HR
    flags['rr_tachy'] = (trends.get('rr_mean_24h', 0) >= HF_RR_TACHY)

    # ---- SBP mean and slope ----
    sbp_series = window_df['sbp'].dropna()
    trends['sbp_mean_24h'] = float(sbp_series.mean()) if len(sbp_series) > 0 else np.nan
    trends['sbp_slope_mmh_per_hr'] = _compute_time_slope(sbp_series.values, window_df.loc[sbp_series.index, 'timestamp'])
    flags['sbp_rising_trend'] = False
    if not np.isnan(trends['sbp_slope_mmh_per_hr']):
        sbp_slope_per_24h = trends['sbp_slope_mmh_per_hr'] * 24.0
        trends['sbp_slope_mmh_per_24h'] = float(round(sbp_slope_per_24h, 3))
        flags['sbp_rising_trend'] = sbp_slope_per_24h >= HF_SBP_RISE_SLOPE_24H

    # ---- Hypotension safety override ----
    recent_6h = df[df['timestamp'] >= now - pd.Timedelta(hours=6)]
    flags['sbp_hypotension'] = False
    if 'sbp' in recent_6h.columns and not recent_6h['sbp'].dropna().empty:
        flags['sbp_hypotension'] = recent_6h['sbp'].min() < HF_SBP_HYPOTENSION

    # ---- Shock Index critical check ----
    flags['shock_index_critical'] = False
    if 'shock_index' in window_df.columns:
        if window_df['shock_index'].dropna().max() >= THRESH_SHOCK_INDEX_HIGH:
            flags['shock_index_critical'] = True
    else:
        if all(col in window_df.columns for col in ['heart_rate', 'sbp']):
            hr_values = window_df['heart_rate'].dropna().values
            sbp_values = window_df['sbp'].dropna().values
            min_len = min(len(hr_values), len(sbp_values))
            if min_len > 0:
                hr_values = hr_values[:min_len]
                sbp_values = sbp_values[:min_len]
                si_vals = hr_values / np.clip(sbp_values, 1, None)
                if si_vals.max() >= THRESH_SHOCK_INDEX_HIGH:
                    flags['shock_index_critical'] = True

    # ---- Safety overrides: emergent RR + low SpO2 combo ----
    flags['rr_emergent_with_low_spo2'] = False
    rr_clean = df['resp_rate'].dropna().reset_index(drop=True) if 'resp_rate' in df.columns else pd.Series()
    spo2_clean = df['spo2'].dropna().reset_index(drop=True) if 'spo2' in df.columns else pd.Series()
    
    if not rr_clean.empty and not spo2_clean.empty:
        recent_rr_now = rr_clean.iloc[-1]
        recent_spo2_now = spo2_clean.iloc[-1]
        flags['rr_emergent_with_low_spo2'] = (recent_rr_now >= HF_RR_EMERGENT) and (recent_spo2_now < HF_SPO2_LOW)

    # ---- Compose score ----
    score = 0
    for k, wt in HF_SCORE_WEIGHTS.items():
        if flags.get(k, False):
            score += wt

    # ---- Map score to action ----
    action = None
    for thresh, act in HF_SCORE_ACTIONS:
        if score >= thresh:
            action = act
            break
    if action is None:
        action = HF_SCORE_ACTIONS[-1][1]

    # Safety override bump
    if flags.get('spo2_critical_sustained') or flags.get('sbp_hypotension') or flags.get('rr_emergent_with_low_spo2'):
        action = "URGENT: immediate escalation / ED contact"
        score = max(score, 8)  # Ensure urgent threshold

    # ---- Build summary ----
    summary_parts = []
    if flags.get('rr_slope'): summary_parts.append(f"RR slope {round(trends.get('rr_slope_bph',0),3)} bpm/hr")
    if flags.get('rr_tachy'): summary_parts.append(f"RR {trends.get('rr_mean_24h', '?')} bpm")
    if flags.get('spo2_burden'): summary_parts.append(f"SpO2 <{HF_SPO2_LOW} for {int(trends['spo2_burden92_frac_24h']*100)}% (24h)")
    if flags.get('sbp_rising_trend'): summary_parts.append(f"SBP rising {trends.get('sbp_slope_mmh_per_24h', '?')} mmHg/24h")
    if flags.get('spo2_critical_sustained'): summary_parts.append(f"SpO2 ≤ {HF_SPO2_CRITICAL} sustained")
    if flags.get('sbp_hypotension'): summary_parts.append("Recent SBP < 90 mmHg")
    if flags.get('shock_index_critical'): summary_parts.append("Elevated shock index")

    summary = " ; ".join(summary_parts) if summary_parts else "No HF decompensation signals in window."

    return {
        "flags": flags,
        "trends": trends,
        "score": int(score),
        "action": action,
        "summary": summary
    }

In [25]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Create a timestamp index for the last 36 hours
now = datetime.now()
timestamps = [now - timedelta(hours=h) for h in range(36, 0, -1)]  # 36 hours of data, most recent last

# Create sample data for a decompensating patient
sample_data = {
    'timestamp': timestamps,
    'spo2': [96, 95, 95, 94, 94, 93, 93, 92, 92, 91, 91, 90,  # First 12 hrs: slowly dropping
             90, 90, 89, 89, 89, 88, 88, 88, 87, 87, 87, 86,  # Next 12 hrs: getting worse
             98, 98, 100, 99, 100, 98, 100, 99, 100, 100, 99, 98], # Last 12 hrs: critical
    'resp_rate': [18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23,  # RR steadily increasing
                  24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29,
                  30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35],
    'heart_rate': [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,  # HR increasing with RR
                   84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95,
                   96, 97, 100, 110, 100, 101, 102, 103, 104, 105, 106, 107],
    'sbp': [118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,  # SBP slowly rising
            130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
            142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153],
    'dbp': [72, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77,  # DBP also rising
            78, 78, 79, 79, 80, 80, 81, 81, 82, 82, 83, 83,
            84, 84, 85, 85, 86, 86, 87, 87, 88, 88, 89, 89],
    'temperature': [36.8] * 36,  # Normal temp
    'weight_kg': [80.0, 80.1, 80.3, 80.4, 80.6, 80.7, 80.9, 81.0, 81.2, 81.3, 81.5, 81.6,  # Weight gain >2kg/24h
                  81.8, 81.9, 82.1, 82.2, 82.4, 82.5, 82.7, 82.8, 83.0, 83.1, 83.3, 83.4,
                  83.6, 83.7, 83.9, 84.0, 84.2, 84.3, 84.5, 86, 86, 85, 85.2, 86.1],
    'age': [67] * 36  # Geriatric patient
} 

# Create the DataFrame (same as before, but we won't use weight)
df_patient = pd.DataFrame(sample_data)

# Run your HF assessment function (NO weight_col parameter)
result = hf_decompensation_assessment(df_patient, window_hours=24)

# Print the results
print(f"Risk Score: {result['score']}")
print(f"Alert Level: {result['action']}")
print(f"Summary: {result['summary']}")

print("\nTRENDS")
for key, value in result['trends'].items():
    if value is not None and not key.startswith('weight'):  # Skip weight trends
        print(f"{key}: {value}")

print("\nFLAGS")
for key, value in result['flags'].items():
    if value and not key.startswith('weight'):  # Skip weight flags
        print(f"{key}: {value}")

Risk Score: 11
Alert Level: URGENT: immediate escalation / ED contact
Summary: RR 29.24 bpm ; SpO2 <92 for 52% (24h) ; SBP rising 24.0 mmHg/24h ; SpO2 ≤ 88 sustained

TRENDS
spo2_burden92_frac_24h: 0.52
rr_mean_24h: 29.24
rr_slope_bph: 0.4999999999999997
sbp_mean_24h: 141.0
sbp_slope_mmh_per_hr: 1.0000000000000004
sbp_slope_mmh_per_24h: 24.0

FLAGS
spo2_burden: True
spo2_critical_sustained: True
rr_tachy: True
sbp_rising_trend: True


In [26]:
result = hf_decompensation_assessment(df_patient)
print(result)

{'flags': {'spo2_burden': True, 'spo2_critical_sustained': True, 'rr_slope': False, 'rr_tachy': True, 'sbp_rising_trend': True, 'sbp_hypotension': np.False_, 'shock_index_critical': False, 'rr_emergent_with_low_spo2': np.False_}, 'trends': {'spo2_burden92_frac_24h': 0.52, 'rr_mean_24h': 29.24, 'rr_slope_bph': 0.4999999999999997, 'sbp_mean_24h': 141.0, 'sbp_slope_mmh_per_hr': 1.0000000000000004, 'sbp_slope_mmh_per_24h': 24.0}, 'score': 11, 'action': 'URGENT: immediate escalation / ED contact', 'summary': 'RR 29.24 bpm ; SpO2 <92 for 52% (24h) ; SBP rising 24.0 mmHg/24h ; SpO2 ≤ 88 sustained'}


In [ ]:
def compute_recent_trends_delta(df):
    """
    Computes trends for each vital by differencing consecutive readings.
    Applies stricter interpretation using age-specific thresholds.
    """
    df = df.copy().sort_values("timestamp").reset_index(drop=True)

    if 'age_category' not in df.columns:
        df = assign_age_category(df)

    trends = {}
    recent = df.tail(TREND_WINDOW)
    age_group = recent['age_category'].iloc[-1]
    thresholds = AGE_THRESHOLDS[age_group]

    vital_map = {
        'resp_rate': ('rr_low', 'rr_normal', 'rr_high'),
        'heart_rate': ('hr_low', 'hr_normal', 'hr_high'),
        'sbp': ('sbp_low', 'sbp_normal', 'sbp_high'),
        'temperature': ('temp_low', 'temp_normal', 'temp_high'),
        'spo2': (None, None, None)  # handled separately
    }

    for vital in ['resp_rate', 'heart_rate', 'sbp', 'temperature', 'spo2']:
        if vital not in recent.columns or recent[vital].isnull().all():
            continue

        y = recent[vital].dropna().values
        if len(y) < 2:
            continue

        avg_delta = np.mean(np.diff(y))
        latest = y[-1]
        trends[f"{vital}_trend"] = round(avg_delta, 3)

        if vital == 'spo2':
            if latest < THRESH_SPO2_LOW:
                if avg_delta > 0:
                    flag = "Still abnormal — but improving"
                elif avg_delta < 0:
                    flag = "Abnormal and worsening"
                else:
                    flag = "Abnormal and flat"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        else:
            low_key, norm_key, high_key = vital_map[vital]
            low = thresholds[low_key]
            normal = thresholds[norm_key]
            high = thresholds[high_key]

            if latest < low or latest > high:
                if (latest > high and avg_delta < 0) or (latest < low and avg_delta > 0):
                    flag = "Still abnormal — but improving"
                else:
                    flag = "Abnormal and worsening"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        trends[f"{vital}_trend_flag"] = flag

    # Shock Index trend
    if all(col in recent.columns for col in ['heart_rate', 'sbp']):
        hr = recent['heart_rate'].values
        sbp = np.clip(recent['sbp'].values, a_min=1, a_max=None)
        si = hr / sbp

        if len(si) >= 2:
            avg_si_delta = np.mean(np.diff(si))
            trends['shock_index_trend'] = round(avg_si_delta, 3)

            latest_si = si[-1]
            if latest_si >= THRESH_SHOCK_INDEX_HIGH:
                flag = "Shock Index high — improving" if avg_si_delta < 0 else "Shock Index high — worsening"
            else:
                flag = "Normal but improving" if avg_si_delta < 0 else "Normal but rising"

            trends['shock_index_trend_flag'] = flag

    return trends

In [ ]:
# main HF pipeline function
def hf_decompensation_assessment(df_patient,
                                 window_hours=24,
                                 weight_col='weight_kg'):
    """
    Assess HF decompensation for a single patient's timeseries DataFrame.
    """
    df = _ensure_sorted_index(df_patient)
    now = df['timestamp'].iloc[-1]

    # subset window
    window_start = now - pd.Timedelta(hours=window_hours)
    window_df = df[df['timestamp'] >= window_start].copy()
    if window_df.empty:
        raise ValueError("No data in the lookback window to compute HF assessment.")

    # ensure age_category exists
    if 'age_category' not in df.columns:
        try:
            df = assign_age_category(df)
            window_df = df[df['timestamp'] >= window_start]
        except Exception:
            df['age_category'] = 'adult'
            window_df['age_category'] = 'adult'

    flags = {}
    trends = {}

    # Weight gain 24h
    flags['weight_gain_24h'] = False
    if weight_col in df.columns:
        # FIX: Reset index to avoid KeyError
        weight_data = df.dropna(subset=[weight_col]).reset_index(drop=True)
        if not weight_data.empty:
            latest_weight = weight_data.iloc[-1][weight_col]  # Get last weight
            # find weight at (now - 24h) as nearest earlier measurement
            t24 = now - pd.Timedelta(hours=24)
            earlier = df[df['timestamp'] <= t24].dropna(subset=[weight_col])
            if not earlier.empty:
                # Reset index to avoid KeyError
                earlier = earlier.reset_index(drop=True)
                weight_24h = earlier.iloc[-1][weight_col]
                delta_w = latest_weight - weight_24h
                trends['weight_delta_24h_kg'] = float(round(delta_w, 3))
                flags['weight_gain_24h'] = trends['weight_delta_24h_kg'] >= HF_WEIGHT_GAIN_KG_24H

    # SpO2 burden & critical sustained
    spo2_series = window_df['spo2'].dropna()
    trends['spo2_burden92_frac_24h'] = _percent_below(spo2_series, HF_SPO2_LOW)
    flags['spo2_burden'] = trends['spo2_burden92_frac_24h'] >= HF_SPO2_BURDEN_PCT_24H
    flags['spo2_critical_sustained'] = _sustained_low_event(window_df['spo2'], window_df['timestamp'],
                                                             HF_SPO2_CRITICAL, HF_SPO2_CRITICAL_SUSTAINED_MINUTES)

    # RR mean and slope
    rr_series = window_df['resp_rate'].dropna()
    trends['rr_mean_24h'] = float(rr_series.mean()) if len(rr_series) > 0 else np.nan
    trends['rr_slope_bph'] = _compute_time_slope(rr_series.values, window_df.loc[rr_series.index, 'timestamp'])
    flags['rr_slope'] = not np.isnan(trends['rr_slope_bph']) and trends['rr_slope_bph'] >= HF_RR_SLOPE_BPM_PER_HR
    flags['rr_tachy'] = (trends.get('rr_mean_24h', 0) >= HF_RR_TACHY)

    # SBP mean and slope
    sbp_series = window_df['sbp'].dropna()
    trends['sbp_mean_24h'] = float(sbp_series.mean()) if len(sbp_series) > 0 else np.nan
    trends['sbp_slope_mmh_per_hr'] = _compute_time_slope(sbp_series.values, window_df.loc[sbp_series.index, 'timestamp'])
    flags['sbp_rising_trend'] = False
    if not np.isnan(trends['sbp_slope_mmh_per_hr']):
        sbp_slope_per_24h = trends['sbp_slope_mmh_per_hr'] * 24.0
        trends['sbp_slope_mmh_per_24h'] = float(round(sbp_slope_per_24h, 3))
        flags['sbp_rising_trend'] = sbp_slope_per_24h >= HF_SBP_RISE_SLOPE_24H

    # hypotension safety override
    recent_6h = df[df['timestamp'] >= now - pd.Timedelta(hours=6)]
    flags['sbp_hypotension'] = False
    if 'sbp' in recent_6h.columns and not recent_6h['sbp'].dropna().empty:
        flags['sbp_hypotension'] = recent_6h['sbp'].min() < HF_SBP_HYPOTENSION

    # Shock Index critical check
    flags['shock_index_critical'] = False
    if 'shock_index' in window_df.columns:
        if window_df['shock_index'].dropna().max() >= THRESH_SHOCK_INDEX_HIGH:
            flags['shock_index_critical'] = True
    else:
        if all(col in window_df.columns for col in ['heart_rate', 'sbp']):
            # FIX: Use .values directly from the Series to avoid index issues
            hr_values = window_df['heart_rate'].dropna().values
            sbp_values = window_df['sbp'].dropna().values
            # Use the shorter length to align arrays
            min_len = min(len(hr_values), len(sbp_values))
            if min_len > 0:
                hr_values = hr_values[:min_len]
                sbp_values = sbp_values[:min_len]
                si_vals = hr_values / np.clip(sbp_values, 1, None)
                if si_vals.max() >= THRESH_SHOCK_INDEX_HIGH:
                    flags['shock_index_critical'] = True

    # Safety overrides: emergent RR + low SpO2 combo
    flags['rr_emergent_with_low_spo2'] = False
    # FIX: Reset index before using iloc
    rr_clean = df['resp_rate'].dropna().reset_index(drop=True) if 'resp_rate' in df.columns else pd.Series()
    spo2_clean = df['spo2'].dropna().reset_index(drop=True) if 'spo2' in df.columns else pd.Series()
    
    if not rr_clean.empty and not spo2_clean.empty:
        recent_rr_now = rr_clean.iloc[-1]
        recent_spo2_now = spo2_clean.iloc[-1]
        flags['rr_emergent_with_low_spo2'] = (recent_rr_now >= HF_RR_EMERGENT) and (recent_spo2_now < HF_SPO2_LOW)

    # Compose score
    score = 0
    for k, wt in HF_SCORE_WEIGHTS.items():
        if flags.get(k, False):
            score += wt

    # Map score to action
    action = None
    for thresh, act in HF_SCORE_ACTIONS:
        if score >= thresh:
            action = act
            break
    if action is None:
        action = HF_SCORE_ACTIONS[-1][1]

    # Safety override bump
    if flags.get('spo2_critical_sustained') or flags.get('sbp_hypotension') or flags.get('rr_emergent_with_low_spo2'):
        action = "URGENT: immediate escalation / ED contact"
        score = max(score, HF_SCORE_WEIGHTS.get('spo2_critical_sustained', 4))

    # Build short summary
    summary_parts = []
    if flags.get('weight_gain_24h'): summary_parts.append(f"Weight +{trends.get('weight_delta_24h_kg','?')} kg (24h)")
    if flags.get('rr_slope'): summary_parts.append(f"RR slope {round(trends.get('rr_slope_bph',0),3)} bpm/hr")
    if flags.get('spo2_burden'): summary_parts.append(f"SpO2 <{HF_SPO2_LOW} for {int(trends['spo2_burden92_frac_24h']*100)}% (24h)")
    if flags.get('sbp_rising_trend'): summary_parts.append(f"SBP rising {trends.get('sbp_slope_mmh_per_24h', '?')} mmHg/24h")
    if flags.get('spo2_critical_sustained'): summary_parts.append(f"SpO2 ≤ {HF_SPO2_CRITICAL} sustained")
    if flags.get('sbp_hypotension'): summary_parts.append("Recent SBP < 90 mmHg")

    summary = " ; ".join(summary_parts) if summary_parts else "No HF decompensation signals in window."

    return {
        "flags": flags,
        "trends": trends,
        "score": int(score),
        "action": action,
        "summary": summary
    }